In [6]:
# MtE for the EFIE for the sphere 


In [1]:
import bempp.api 
import time
import numpy as np
from bempp.api.operators.boundary.sparse import identity
from bempp.api.assembly.blocked_operator import BlockedOperator, GeneralizedBlockedOperator

bempp.api.enable_console_logging()
k = 15
precision = 10

h = 2 * np.pi/(precision*k)
print(h, ': h')

grid = bempp.api.shapes.sphere(h=h)
rwg = bempp.api.function_space(grid, 'RWG', 0)
rwg = bempp.api.function_space(grid, 'RWG', 0)
snc = bempp.api.function_space(grid, 'SNC', 0)
p1d = bempp.api.function_space(grid, 'DP', 1)

def incident_field(x):
    return np.array([np.exp(1j * k * x[2]), 0. * x[2], 0. * x[2]])

@bempp.api.complex_callable
def tangential_trace(x, n, domain_index, result):
    incident_field = np.array([np.exp(1j * k * x[2]), 0. * x[2], 0. * x[2]])
    result[:] = np.cross(incident_field, n)

trace_fun = bempp.api.GridFunction(rwg, fun=tangential_trace, dual_space=snc)
trace_fun_proj = trace_fun.projections()

print('assembling efie_wf')
ta = time.time()
efie = bempp.api.operators.boundary.maxwell.electric_field(rwg,rwg,snc,k)
efie_wf = efie.weak_form()
print(time.time()- ta, 'ta efie_wf')

print('assembling efie_M')
ta = time.time()
efie_M = bempp.api.as_matrix(efie_wf)
print(time.time()- ta, 'ta efie_M')

print('assembling mte_wf')
ta = time.time()
mte = bempp.api.operators.boundary.maxwell.osrc_mte( [snc,p1d],  [snc, p1d],  [snc, p1d], k)
mte_wf = mte._assemble()
print(time.time()- ta, 'ta mte_wf')



0.041887902047863905 : h


bempp:HOST:INFO: Created grid with id 142c7dac-3dac-44f7-91ef-152d7f3a2c63. Elements: 17620. Edges: 26430. Vertices: 8812
bempp:HOST:INFO: OpenCL CPU Device set to: pthread-AMD EPYC 7302 16-Core Processor


assembling efie_wf
20.49252486228943 ta efie_wf
assembling efie_M
6.079673767089844e-05 ta efie_M
assembling mte_wf
23.724127769470215 ta mte_wf


In [2]:
class gmres_counter(object):
    def __init__(self, disp=True):
        self._disp = disp
        self.niter = 0
        self.residual = []
    def __call__(self, rk=None):
        self.niter += 1
        self.residual.append(rk)
        if self._disp:
            print('iter %3i\trk = %s' % (self.niter, str(rk)))


In [ ]:
from scipy.sparse.linalg import gmres as sc_gmres
maxiter = 1000
counter_gmres = gmres_counter()
counter_prec = gmres_counter()


print('direct solver')
ts = time.time()
lambda_data_lu = np.linalg.solve(efie_M, trace_fun_proj)
ts_M = time.time()-ts
print(ts_M, 'ts efie_M')

print('unpreconditioned gmres')
ts = time.time()
lambda_data_gmres, conv_gmres = sc_gmres(efie_wf, trace_fun.projections(), maxiter=maxiter, callback=counter_gmres, restart=maxiter)
ts_gmres = time.time()-ts
print(ts_gmres, 'ts efie_wf')

print('MtE preconditioned gmres')
ts = time.time()
lambda_data_prec, conv_gmres = sc_gmres(mte_wf * efie_wf, mte_wf * trace_fun.projections(), maxiter=maxiter, callback=counter_prec, restart=maxiter)
ts_prec = time.time() - ts
print(ts_prec, 'ts efie_prec')

direct solver


In [ ]:
print(ts_M, 'ts efie_M')
print(ts_gmres, 'ts efie_wf')
print(ts_prec, 'ts efie_prec')

In [ ]:
from matplotlib import pyplot as plt
plt.plot(np.abs(lambda_data_lu), label='lu')
plt.plot(np.abs(lambda_data_gmres), label='gmres')
plt.plot(np.abs(lambda_data_prec), label='prec')
plt.legend()